In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

훈련하는 동안 모델 내부에서 일어나는 일을 조사하고 제어하는 방법을 살펴보겠습니다.
* 대규모 데이터셋에서 `model.fit()`이나 `model.fit_generator()`로 수십 번의 에포크를 실행하는 것은 **종이 비행기**를 날리는 것과 비슷합니다.
* 손을 떠나면 종이 비행기 경로와 착륙 지점을 제어할 방법이 없습니다
* 나쁜 결과를 피하려면 종이 비행기 대신 **드론**을 사용하는 것이 좋습니다
* 앞으로 소개할 기법은 `model.fit()` 호출을 종이 비행기에서 스스로 판단하고 동적으로 결정하는 똑똑한 자동 드론으로 바꾸어 줄 것입니다

### 콜백을 사용하여 모델의 훈련 과정 제어하기